In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from src.nhl_scraper import time_remaining, nhl_scraper, shot_scraper2, get_processed_data, get_skater_stats, add_skaters_on_ice, get_corsi_df, get_toi_df, get_attempts_df

In [3]:
import pandas as pd
from nhlpy import NHLClient
from datetime import datetime

In [4]:
game_id = 2024021036

In [5]:
shots_df,blocks_df,misses_df,goals_df,hits_df,give_take_df,faceoffs_df,pens_df,shifts_df,players_df,games_df,teams_df = nhl_scraper([game_id])

In [6]:
from src.config import MODELS_DIR
import joblib

model = joblib.load(MODELS_DIR / "xgb_v1.pkl")

/home/nolan/anaconda3/envs/reports/lib/python3.13/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/nolan/anaconda3/envs/reports/lib/python3.13/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/nolan/anaconda3/envs/reports/lib/python3.13/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.6

In [7]:
temp_df, time_df = shot_scraper2([game_id])
temp_df = get_skater_stats(temp_df)
processed_df = get_processed_data(temp_df)

processed_df

,home,last_play,rebound,rush,home_skaters,away_skaters,x_coord,y_coord,shot_type,zone,position,shooter_hand,shooting_pct,glove_hand,save_pct,angle,shot_on_glove,situation,target
0,Home,hit,No rebound,No rush,5,5,44,39,wrist,O,D,R,0.0464,L,0.898198,40.91,RL,EV,0
1,Home,faceoff,No rebound,No rush,5,5,78,13,snap,O,R,L,0.1260,L,0.898198,49.76,LL,EV,0
2,Home,hit,No rebound,No rush,5,5,71,9,wrist,O,L,R,0.1319,L,0.898198,26.57,RL,EV,0
3,Away,shot-on-goal,No rebound,No rush,5,5,69,19,wrist,O,C,L,0.1420,L,0.905881,43.53,LL,EV,0
4,Home,takeaway,No rebound,No rush,5,5,32,11,slap,O,D,L,0.0446,L,0.898198,10.92,LL,EV,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Home,missed-shot,No rebound,No rush,4,4,80,-1,tip-in,O,L,R,0.1456,L,0.898198,-6.34,RL,EV,0
81,Home,faceoff,No rebound,No rush,5,4,51,4,snap,O,D,R,0.0464,L,0.898198,6.01,RL,PP,1
82,Away,faceoff,No rebound,No rush,4,5,41,-13,slap,O,C,L,0.1111,L,0.905881,-15.15,LL,SH,0
83,Away,giveaway,No rebound,Rush,5,5,32,-1,snap,O,D,R,0.0372,L,0.905881,-1.01,RL,EV,0


In [8]:
full_shots_df = add_skaters_on_ice(processed_df, time_df, shifts_df)
full_shots_df.columns

Index(['home', 'last_play', 'rebound', 'rush', 'home_skaters', 'away_skaters',
       'x_coord', 'y_coord', 'shot_type', 'zone', 'position', 'shooter_hand',
       'shooting_pct', 'glove_hand', 'save_pct', 'angle', 'shot_on_glove',
       'situation', 'target', 'period', 'time', 'time_seconds', 'game_seconds',
       'home_players', 'away_players'],
      dtype='object')

In [9]:
preds = model.predict_proba(full_shots_df.drop(['period', 'time', 'time_seconds', 'game_seconds',
       'home_players', 'away_players'], axis=1))
xG = preds[:,1]

In [10]:
full_shots_df["xG"] = xG

In [11]:
full_shots_df

,home,last_play,rebound,rush,home_skaters,away_skaters,x_coord,y_coord,shot_type,zone,...,shot_on_glove,situation,target,period,time,time_seconds,game_seconds,home_players,away_players,xG
0,Home,hit,No rebound,No rush,5,5,44,39,wrist,O,...,RL,EV,0,1,00:43,43,43,"[8475279, 8475852, 8477353, 8477946, 8481542, ...","[8478043, 8479359, 8480045, 8480839, 8481522, ...",0.008904
1,Home,faceoff,No rebound,No rush,5,5,78,13,snap,O,...,LL,EV,0,1,01:39,99,99,"[8475225, 8475718, 8475765, 8475852, 8476979, ...","[8475722, 8477949, 8480045, 8480196, 8480802, ...",0.204576
2,Home,hit,No rebound,No rush,5,5,71,9,wrist,O,...,RL,EV,0,1,01:56,116,116,"[8474141, 8475718, 8475765, 8475852, 8476979, ...","[8475722, 8477949, 8480045, 8480196, 8480802, ...",0.146474
3,Away,shot-on-goal,No rebound,No rush,5,5,69,19,wrist,O,...,LL,EV,0,1,02:06,126,126,"[8474141, 8475718, 8475765, 8475852, 8482762, ...","[8477365, 8478413, 8480045, 8480802, 8481524, ...",0.052182
4,Home,takeaway,No rebound,No rush,5,5,32,11,slap,O,...,LL,EV,0,1,02:57,177,177,"[8474141, 8475279, 8475852, 8479337, 8481542, ...","[8477365, 8478413, 8480045, 8482097, 8482671, ...",0.029260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Home,missed-shot,No rebound,No rush,4,4,80,-1,tip-in,O,...,RL,EV,0,3,13:26,806,3206,"[8475852, 8477946, 8481607, 8482078, 8482762]","[8480045, 8480802, 8480839, 8481524, 8483468]",0.130279
81,Home,faceoff,No rebound,No rush,5,4,51,4,snap,O,...,RL,PP,1,3,14:31,871,3271,"[8474141, 8475225, 8475718, 8475852, 8476979, ...","[8475722, 8477365, 8478043, 8479359, 8480045, ...",0.077886
82,Away,faceoff,No rebound,No rush,4,5,41,-13,slap,O,...,LL,SH,0,3,18:32,1112,3512,"[8475279, 8475852, 8477353, 8481542, 8483464]","[8479359, 8480045, 8480802, 8480839, 8481522, ...",0.028868
83,Away,giveaway,No rebound,Rush,5,5,32,-1,snap,O,...,RL,EV,0,3,19:15,1155,3555,"[8474141, 8475279, 8475852, 8481542, 8481725, ...","[8477365, 8478043, 8479359, 8480045, 8481522, ...",0.015626


In [25]:
from src.nhl_scraper import tally_xG

In [26]:
tally_xG(full_shots_df, players_df)

,player_id,period,xG_for,xG_against,name,position
0,8481607,1,0.593940,0.037513,Albert Johansson,D
1,8481607,2,0.210643,0.109572,Albert Johansson,D
2,8481607,3,0.922114,0.164356,Albert Johansson,D
3,8479337,1,0.729466,0.146118,Alex DeBrincat,R
4,8479337,2,0.834753,0.354618,Alex DeBrincat,R
...,...,...,...,...,...,...
109,8475765,2,0.451430,0.328987,Vladimir Tarasenko,R
110,8475765,3,0.502552,0.163988,Vladimir Tarasenko,R
111,8484145,1,0.409235,0.656757,Zach Benson,L
112,8484145,2,0.196494,0.174875,Zach Benson,L


In [12]:
shifts_df

,game,home_id,away_id,shift_team,period,start_time,end_time,duration,player_id
0,2024021036,17,7,17,1,19:07,19:07,None,8474141
1,2024021036,17,7,17,1,01:55,03:15,01:20,8474141
2,2024021036,17,7,17,1,05:54,06:36,00:42,8474141
3,2024021036,17,7,17,1,09:12,09:38,00:26,8474141
4,2024021036,17,7,17,1,10:22,11:11,00:49,8474141
...,...,...,...,...,...,...,...,...,...
793,2024021036,17,7,7,3,05:39,06:26,00:47,8484145
794,2024021036,17,7,7,3,08:26,09:00,00:34,8484145
795,2024021036,17,7,7,3,10:12,10:48,00:36,8484145
796,2024021036,17,7,7,3,15:11,15:32,00:21,8484145


In [13]:
shifts_df.columns

Index(['game', 'home_id', 'away_id', 'shift_team', 'period', 'start_time',
       'end_time', 'duration', 'player_id'],
      dtype='object')

In [14]:
get_toi_df(shifts_df, players_df)

,duration,duration_min
0,308,05:08
1,298,04:58
2,352,05:52
3,318,05:18
4,298,04:58
...,...,...
109,313,05:13
110,298,04:58
111,315,05:15
112,419,06:59


In [15]:
attempts_df = get_attempts_df(shots_df, misses_df, blocks_df, goals_df)

In [16]:
attempts_df

,home_skaters,away_skaters,event_owner,period,time,play_type,shooter_id
0,5,5,17,1,19:17,shot-on-goal,8481542
1,5,5,17,1,19:14,blocked-shot,8482078
2,5,5,17,1,18:21,missed-shot,8475765
3,5,5,17,1,18:15,blocked-shot,8475225
4,5,5,17,1,18:04,shot-on-goal,8477456
...,...,...,...,...,...,...,...
111,4,5,7,3,01:28,shot-on-goal,8483468
112,5,5,7,3,01:06,blocked-shot,8483468
113,5,5,7,3,00:45,missed-shot,8477365
114,4,4,7,3,00:33,blocked-shot,8477365


In [30]:
attempts_by_period = attempts_df.groupby(["period","shooter_id"]).size().reset_index(name="total_attempts")

In [34]:
pd.merge(attempts_by_period, players_df, left_on="shooter_id", right_on="player_id").sort_values(by=["name", "period"], ascending=[True, True]).reset_index(drop=True)

,period,shooter_id,total_attempts,player_id,name,position
0,1,8479337,1,8479337,Alex DeBrincat,R
1,2,8479337,2,8479337,Alex DeBrincat,R
2,3,8479337,1,8479337,Alex DeBrincat,R
3,1,8477949,3,8477949,Alex Tuch,R
4,1,8475279,2,8475279,Ben Chiarot,D
...,...,...,...,...,...,...
65,3,8477353,1,8477353,Tyler Motte,C
66,1,8475765,2,8475765,Vladimir Tarasenko,R
67,2,8475765,3,8475765,Vladimir Tarasenko,R
68,2,8484145,1,8484145,Zach Benson,L


In [50]:
from src.nhl_scraper import fill_shot_attempts

In [52]:
full_attempts_df = fill_shot_attempts(attempts_df, players_df)

In [44]:
import itertools

In [45]:
attempts_by_period = attempts_df.groupby(["period","shooter_id"]).size().reset_index(name="total_attempts")
all_players = attempts_df["shooter_id"].unique()
all_periods = attempts_df["period"].unique()
full_index = pd.DataFrame(itertools.product(all_players, all_periods), columns=["player_id", "period"])

In [46]:
full_index

,player_id,period
0,8481542,1
1,8481542,2
2,8481542,3
3,8482078,1
4,8482078,2
...,...,...
94,8484145,2
95,8484145,3
96,8479992,1
97,8479992,2


In [47]:
pd.merge(full_index, attempts_by_period, left_on=["player_id", "period"], right_on=["shooter_id", "period"], how="left")

,player_id,period,shooter_id,total_attempts
0,8481542,1,8481542.0,1.0
1,8481542,2,8481542.0,2.0
2,8481542,3,8481542.0,2.0
3,8482078,1,8482078.0,1.0
4,8482078,2,8482078.0,3.0
...,...,...,...,...
94,8484145,2,8484145.0,1.0
95,8484145,3,8484145.0,1.0
96,8479992,1,NaN,NaN
97,8479992,2,8479992.0,1.0


In [17]:
from src.nhl_scraper import get_on_ice_players

In [19]:
corsi_df = get_corsi_df(attempts_df=attempts_df, shifts_df=shifts_df)

In [20]:
corsi_df

,home_skaters,away_skaters,event_owner,period,time,play_type,shooter_id,tr_seconds,time_seconds,game_seconds,home_players,away_players
0,5,5,17,1,19:17,shot-on-goal,8481542,1157,43,43,"[8475279, 8475852, 8477353, 8477946, 8481542, ...","[8478043, 8479359, 8480045, 8480839, 8481522, ..."
1,5,5,17,1,19:14,blocked-shot,8482078,1154,46,46,"[8475279, 8475852, 8477946, 8481542, 8481725, ...","[8479420, 8480045, 8480064, 8480807, 8480839, ..."
2,5,5,17,1,18:21,missed-shot,8475765,1101,99,99,"[8475225, 8475718, 8475765, 8475852, 8476979, ...","[8475722, 8477949, 8480045, 8480196, 8480802, ..."
3,5,5,17,1,18:15,blocked-shot,8475225,1095,105,105,"[8475225, 8475718, 8475765, 8475852, 8476979, ...","[8475722, 8477949, 8480045, 8480196, 8480802, ..."
4,5,5,17,1,18:04,shot-on-goal,8477456,1084,116,116,"[8474141, 8475718, 8475765, 8475852, 8476979, ...","[8475722, 8477949, 8480045, 8480196, 8480802, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
111,4,5,7,3,01:28,shot-on-goal,8483468,88,1112,3512,"[8475279, 8475852, 8477353, 8481542, 8483464]","[8479359, 8480045, 8480802, 8480839, 8481522, ..."
112,5,5,7,3,01:06,blocked-shot,8483468,66,1134,3534,"[8474141, 8475279, 8475852, 8477353, 8481542, ...","[8479359, 8480045, 8480802, 8480839, 8481522, ..."
113,5,5,7,3,00:45,missed-shot,8477365,45,1155,3555,"[8474141, 8475279, 8475852, 8481542, 8481725, ...","[8477365, 8478043, 8479359, 8480045, 8481522, ..."
114,4,4,7,3,00:33,blocked-shot,8477365,33,1167,3567,"[8475225, 8475852, 8476979, 8477353, 8481607]","[8477365, 8478043, 8480045, 8481522, 8482671]"


In [21]:
from src.nhl_scraper import tally_corsi

In [22]:
tally_corsi(corsi_df=corsi_df, players_df=players_df, game_id=game_id)

,player_id,period,corsi_for,corsi_against,name,position
0,8481607,1,8,3,Albert Johansson,D
1,8481607,2,5,5,Albert Johansson,D
2,8481607,3,7,6,Albert Johansson,D
3,8479337,1,9,4,Alex DeBrincat,R
4,8479337,2,9,7,Alex DeBrincat,R
...,...,...,...,...,...,...
109,8475765,2,7,2,Vladimir Tarasenko,R
110,8475765,3,3,3,Vladimir Tarasenko,R
111,8484145,1,6,8,Zach Benson,L
112,8484145,2,8,5,Zach Benson,L


In [ ]:
from nhlpy import NHLClient

client = NHLClient()

In [ ]:
client.game_center.boxscore(game_id)

{'id': 2024021036,
 'season': 20242025,
 'gameType': 2,
 'limitedScoring': False,
 'gameDate': '2025-03-12',
 'venue': {'default': 'Little Caesars Arena'},
 'venueLocation': {'default': 'Detroit'},
 'startTimeUTC': '2025-03-12T23:30:00Z',
 'easternUTCOffset': '-04:00',
 'venueUTCOffset': '-04:00',
 'tvBroadcasts': [{'id': 385,
   'market': 'N',
   'countryCode': 'US',
   'network': 'TNT',
   'sequenceNumber': 11},
  {'id': 501,
   'market': 'N',
   'countryCode': 'US',
   'network': 'truTV',
   'sequenceNumber': 14},
  {'id': 519,
   'market': 'N',
   'countryCode': 'US',
   'network': 'MAX',
   'sequenceNumber': 18},
  {'id': 281,
   'market': 'N',
   'countryCode': 'CA',
   'network': 'TVAS',
   'sequenceNumber': 109},
  {'id': 562,
   'market': 'H',
   'countryCode': 'US',
   'network': 'FDSNDET',
   'sequenceNumber': 271}],
 'gameState': 'OFF',
 'gameScheduleState': 'OK',
 'periodDescriptor': {'number': 3,
  'periodType': 'REG',
  'maxRegulationPeriods': 3},
 'regPeriods': 3,
 'awa

In [57]:
get_toi_df(shifts_df, players_df)

,name,period,duration,duration_min
0,Albert Johansson,1,308,05:08
1,Albert Johansson,2,298,04:58
2,Albert Johansson,3,352,05:52
3,Alex DeBrincat,1,318,05:18
4,Alex DeBrincat,2,298,04:58
...,...,...,...,...
109,Vladimir Tarasenko,2,313,05:13
110,Vladimir Tarasenko,3,298,04:58
111,Zach Benson,1,315,05:15
112,Zach Benson,2,419,06:59


In [58]:
give_take_df

,game,home_id,home_team,away_id,away_team,event_owner,period,period_type,time,event_id,last_play,play_type,away_goalie,home_goalie,away_skaters,home_skaters,player_id,x,y,zone
0,2024021036,17,DET,7,BUF,17,1,REG,19:11,116,blocked-shot,giveaway,1,1,5,5,8475279,-63,-41,O
1,2024021036,17,DET,7,BUF,7,1,REG,17:42,141,shot-on-goal,takeaway,1,1,5,5,8482671,-22,16,N
2,2024021036,17,DET,7,BUF,17,1,REG,16:59,146,shot-on-goal,giveaway,1,1,5,5,8479337,-85,-34,O
3,2024021036,17,DET,7,BUF,7,1,REG,14:41,195,shot-on-goal,giveaway,1,1,5,5,8475722,68,-40,O
4,2024021036,17,DET,7,BUF,17,1,REG,13:49,208,hit,giveaway,1,1,5,5,8479337,-4,-6,N
5,2024021036,17,DET,7,BUF,17,1,REG,13:39,213,missed-shot,giveaway,1,1,5,5,8479337,88,-32,D
6,2024021036,17,DET,7,BUF,17,1,REG,11:07,237,blocked-shot,giveaway,1,1,5,5,8475765,-27,41,O
7,2024021036,17,DET,7,BUF,7,1,REG,10:40,238,giveaway,giveaway,1,1,5,5,8482097,-3,38,N
8,2024021036,17,DET,7,BUF,17,1,REG,07:40,324,hit,takeaway,1,1,5,5,8479337,-85,19,O
9,2024021036,17,DET,7,BUF,17,1,REG,07:06,344,shot-on-goal,giveaway,1,1,5,5,8481725,62,-40,D
